# Scraping Amazon Customer Reviews -- Selenium + BeautifulSoup 

In [12]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import re

In [13]:
#Get Chrome driver
driver = webdriver.Chrome(ChromeDriverManager().install())

#Firefox 
#driver = webdriver.Firefox()
#driver.get(urls)

<ipython-input-13-e09616404291>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [14]:
# creat a loop for getting the url for each page
for i in range(1,17):
    urls = "https://www.amazon.com/Alienware-Aurora-R10-Gaming-Desktop/product-reviews/B09VR4G423/ref=cm_cr_arp_d_paging_btm_next_"+str(i)+"?ie=UTF8&reviewerType=all_reviews&pageNumber="+str(i)
    #print(urls)

In [15]:
#store the data into dataframe
AZ_reviews = {'Customer': [], 'Ratings': [], 'Date':[], 'Review_text':[], 'Vertified': []} #initialize dictionary

# creat a loop for getting the url for each page
for i in range(1,3):
    urls = "https://www.amazon.com/Monoprice-3-Feet-mini-B-28AWG-103896/product-reviews/B003L18SHC/ref=cm_cr_arp_d_paging_btm_next_" + str(i) + "?ie=UTF8&reviewerType=all_reviews&pageNumber=" + str(i)
    page = driver.get(urls)
    driver.implicitly_wait(10) #sleep for 10 seconds
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    results = soup.find("div", {"id": "cm_cr-review_list"})
    details = results.find_all('div', class_='a-section celwidget')
    
    #a loop for querying details from the webpage
    for detail in details:
        cus_name = detail.find('span', class_='a-profile-name')
        ratings = detail.find('span', class_="a-icon-alt")
        date = detail.find('span', {"data-hook": "review-date"})
        review = detail.find('span', {"data-hook": "review-body"})
        vertified = detail.find('span', class_='a-link-normal')
        
        
        cus_clean = re.sub(r"[^a-zA-Z0-9]+",'',cus_name.text)
        #put a space in " " otherwise all the words will merge together
        date_clean = re.sub(r"[^a-zA-Z0-9]+",' ',date.text)
        review_clean = re.sub(r"[^a-zA-Z0-9]+",' ',review.text)
        
        AZ_reviews['Customer'].append(cus_clean.strip())
        AZ_reviews['Ratings'].append(ratings.text.strip())
        AZ_reviews['Date'].append(date_clean.strip())
        AZ_reviews['Review_text'].append(review_clean.strip())
        AZ_reviews['Vertified'].append(vertified)
        
        print(cus_clean)
        print(ratings.text.strip())
        print(date_clean)
        print(review_clean)
        print(vertified)
        print()


WilliamIDube
5.0 out of 5 stars
Reviewed in the United States on September 6 2022
 PROS This cable is a great price and works It is a true USB Mini B on one side and Male USB A on the other side It works Nice and long to reach anything I need to be charging or transferring data with CONS None to note Yes I d buy this again and recommend to anyone that needs one 
None

PriscillaSpradling
4.0 out of 5 stars
Reviewed in the United States on September 11 2022
 I bought as part of a bundle for two chords I needed one short cord and one long cord which is what I got and both of the cords are perfect for what I need them for 
None

JHirsh
5.0 out of 5 stars
Reviewed in the United States on September 10 2022
 Needed a cable when the damn gnomes stole one and this worked Has now worked for three separate devices chargers and survived a vacation No issues here 
None

chadm
5.0 out of 5 stars
Reviewed in the United States on September 10 2022
 Product exactly what it should I always prefer the br

In [16]:
AZreviews_df = pd.DataFrame(AZ_reviews)
AZreviews_df.head()

,Customer,Ratings,Date,Review_text,Vertified
0,WilliamIDube,5.0 out of 5 stars,Reviewed in the United States on September 6 2022,PROS This cable is a great price and works It ...,None
1,PriscillaSpradling,4.0 out of 5 stars,Reviewed in the United States on September 11 ...,I bought as part of a bundle for two chords I ...,None
2,JHirsh,5.0 out of 5 stars,Reviewed in the United States on September 10 ...,Needed a cable when the damn gnomes stole one ...,None
3,chadm,5.0 out of 5 stars,Reviewed in the United States on September 10 ...,Product exactly what it should I always prefer...,None
4,JohnJ,5.0 out of 5 stars,Reviewed in the United States on August 29 2022,The price of this cord was beyond phenomenal c...,None


In [5]:
AZreviews_df.info() 
#if you see the following error:
#TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type
#you need to update your Pandas and Numpy, you need to have pandas>=1.0.5
#anaconda prompt: conda update pandas/numpy
#check the version of your pandas: conda list pandas/numpy

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer     39 non-null     object
 1   Ratings      39 non-null     object
 2   Date         39 non-null     object
 3   Review_text  39 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


# Sentiment Analysis 2: Patten

install Patten

https://anaconda.org/conda-forge/pattern

## NLP: Introduction to the Pattern Library

source: https://stackabuse.com/python-for-nlp-introduction-to-the-pattern-library/

In [5]:
from pattern3.web import Twitter

twitter = Twitter()

for results in twitter.search('movies'):
    print(results.url)
    print(results.text)

TypeError: Unicode-objects must be encoded before hashing

In [17]:
from pattern.en import pluralize, singularize

print(pluralize('leaf'))
print(singularize('theives'))

leaves
theife


In [18]:
from pattern.en import comparative, superlative

print(comparative('good'))
print(superlative('good'))

better
best


In [8]:
from pattern.en import sentiment

print(sentiment("This is an excellent movie to watch. I really love it"))

(0.75, 0.8)


In Pattern, the sentiment object is used to find the **polarity** (positivity or negativity) of a text along with its subjectivity.

**Polarity**:Depending upon the most commonly occurring positive (good, best, excellent, etc.) and negative (bad, awful, pathetic, etc.) adjectives, a sentiment score **between 1 and -1** is assigned to the text.

In addition to the sentiment score, subjectivity is also returned. **The subjectivity value can be between 0 and 1.** Subjectivity quantifies the amount of personal opinion and factual information contained in the text. **The higher subjectivity means that the text contains personal opinion rather than factual information.**

##### The sentence "This is an excellent movie to watch. I really love it" has a sentiment of 0.75, which shows that it is highly positive. 
##### Similarly, the subjectivity of 0.8 refers to the fact that the sentence is a personal opinion of the user.

In [6]:
type(AZ_reviews['Review_text'])

list

In [9]:
print(sentiment("Great machine with excellent cooling handles anything I throw at it using ultra settings on games like Battlefield 4 Titanfall2")) 

(0.9, 0.875)


In [9]:
texts = AZ_reviews['Review_text']

for row in texts:
    print(sentiment(row))

NameError: name 'AZ_reviews' is not defined

# Sentiment Analysis 2: Textblob

install Textblob

https://anaconda.org/conda-forge/textblob

In [8]:
from textblob import TextBlob

In [17]:
texts = AZ_reviews['Review_text']
for row in texts:
    pol = TextBlob(row).sentiment.polarity
    sub = TextBlob(row).sentiment.subjectivity
    print(pol,sub,
          "Positive" if pol > 0 
          else "Neutral" if pol == 0 
          else "Negative")

0.2625 0.5458333333333333 Positive
0.31666666666666665 0.5666666666666667 Positive
0.0 0.0 Neutral
0.33333333333333337 0.375 Positive
0.1825 0.4791666666666667 Positive
0.0 0.0 Neutral
-0.04583333333333334 0.37083333333333335 Negative
0.3333333333333333 0.5 Positive
0.0 0.0 Neutral
0.0 0.0 Neutral
0.8 0.75 Positive
0.6 0.7666666666666666 Positive
0.0 0.0 Neutral
0.4 0.7 Positive
0.8 0.75 Positive
0.22000000000000003 0.56 Positive
0.05 0.15000000000000002 Positive
0.17142857142857146 0.34285714285714286 Positive
0.8 0.75 Positive
0.6 0.8 Positive


# Sentiment Analysis 3: vaderSentiment

install vaderSentiment

https://anaconda.org/conda-forge/vadersentiment

In [18]:
#if you have issues to install vaderSentiment at anaconda prompt
#try this: nltk.download('vader_lexicon')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#note: depending on how you installed (e.g., using source code download versus pip install), you may need to import like this:
#from vaderSentiment import SentimentIntensityAnalyzer

In [19]:
analyzer = SentimentIntensityAnalyzer()

In [22]:
for row in texts:
    vsPol = analyzer.polarity_scores(row)
    #print("{:-<65} {}".format(row, str(vsPol)))
    Pol_result = vsPol['compound']
    print(Pol_result,"Positive" if Pol_result >= 0.05 else "Neutral" if -0.05 < Pol_result < 0.05 
          else "Negative")

0.8762 Positive
0.5719 Positive
-0.1531 Negative
-0.1154 Negative
0.3167 Positive
0.0 Neutral
-0.1027 Negative
0.0 Neutral
0.2732 Positive
0.0 Neutral
0.7579 Positive
0.9042 Positive
0.2732 Positive
0.0 Neutral
0.8834 Positive
0.2732 Positive
0.0 Neutral
0.743 Positive
0.6249 Positive
0.4215 Positive


1. positive sentiment: compound score >= 0.05
2. neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
3. negative sentiment: compound score <= -0.05

**Supplementary materials**

vaderSentiment resources and documentation
https://github.com/cjhutto/vaderSentiment

10 Best Python Libraries for Sentiment Analysis
https://www.unite.ai/10-best-python-libraries-for-sentiment-analysis/

1. Pattern
2. VADER
3. TextBlob
4. spaCy
5. CoreNLP
6. scikit-learn
7. Polyglot
8. PyTorch

# Actions:

### Action 1 - Required: Scrape customer reviews of a product from Amazon

### Action 2 - Required: Describe your dataframe

### Action 3 - Required: Conduct sentiment analysis (using the three methods in this lab) to all the reviews and compare the results (short explainations)

### Action 4 - Optional: Try to count the number of positive reviews, neutral reviews, and negative reviews for each method (using len())